# Models

Mitchell and Lapata (2010) define a general class of composition models defined as
$$ \mathbf{p} = \text{comp}(u, v, R, K) $$
representing the composition $\mathbf{p}$ of two constituents $\mathbf{u}, \mathbf{v}$  which stand in some syntactic relation $R$, where $K$ represents any other information used. 

The models we describe here do not make use of the syntactic relation argument $R$, so let
$$ \text{comp}(u, v, K) = \text{comp}(u, v, -, K) $$

Two simplistic models which leave $R$ empty are the *additive model*
$$ \text{comp}(\mathbf{u}, \mathbf{v}, \langle \mathbf{A}, \mathbf{B}\rangle) = \mathbf{uA} + \mathbf{vB} $$
and the *multiplicative model*
$$ \text{comp}(\mathbf{u}, \mathbf{v},  \mathbf{C}) = (\mathbf{u}\odot\mathbf{v})\mathbf{C} $$

where $\mathbf{u}, \mathbf{v} \in \mathbb{R}^d, \mathbf{A, B, C} \in \mathbb{R}^{d \times d}$. In a neural network model using these composition functions, the matrices $\mathbf{A}, \mathbf{B}, \mathbf{C}$ are trained and the word vectors $\mathbf{u}, \mathbf{v}$ may be given (e.g., by `word2vec`) or may also be trained or tuned by the model in parallel.

Here I discuss two classes of models using Mitchell and Lapata's notation:
1. Recursive neural models which assume trees over vector representations of words
2. Models which represent certain words as matrices

## Neural models with explicit compositional structure

These models assume vector representations of each word in an input sequence, and use RNNs and variants of RNNs to learn composition functions on vectors. Depending on the task, the vector representations of the words may be learned or tuned by the model in parallel.

### Recursive Neural Network (RNN)

An input sequence $\mathbf{x}$ of vectors (representing a sequence of words) is parsed into a binary tree structure.  Given two vectors $\mathbf{a}, \mathbf{b}$, the vector of the parent node $\mathbf{p}$ is the output of the composition function
$$ \mathbf{p} = \text{comp}(\mathbf{a}, \mathbf{b}, K). $$

Socher et al. (2010) give this composition function:
$$ \text{comp}(\mathbf{a}, \mathbf{b}, \mathbf{W}) = f\left(\begin{bmatrix}\mathbf{a}\\\mathbf{b}\end{bmatrix}\mathbf{W}\right) $$
where $\mathbf{a}, \mathbf{b} \in \mathbb{R}^d, \mathbf{W} \in \mathbb{R}^{2d\times d}$. $\mathbf{W}$ is the parameter to learn and is fixed for all compositions of $\mathbf{a}$ and $\mathbf{b}$. $f$ is the activation function, usually $\tanh$. The model may also include a bias term, not shown above.

<img src="images/socher2010fig1.png" />

For a classification task, the softmax classifier is applied either to the root node or to each node $\mathbf{a}$ in the tree (dependending on the structure of your training data)
$$ \mathbf{y^{(a)}} = \text{softmax}(\mathbf{a}\mathbf{W}_s) $$
where $\mathbf{W}_s \in \mathbb{R}^{d\times m}$ for an $m$-way classification task.

### Matrix-Vector RNN (MV-RNN)

An extension of an RNN, so that there can be a different composition function for each pair of children. Given two vectors $\mathbf{a}, \mathbf{b}$ they are associated respectively with the matrices $\mathbf{A}, \mathbf{B}$. So each node is represented by a pair of a vector and a matrix.

<img src="images/socher2012fig2.png" />

The composition function is
$$ \text{comp}(\langle\mathbf{a}, \mathbf{A}\rangle, \langle\mathbf{b}, \mathbf{B}\rangle,  \mathbf{W}) = 
\left\langle f\left(\begin{bmatrix}\mathbf{Ba}\\\mathbf{Ab}\end{bmatrix}\mathbf{W}\right),
\begin{bmatrix}\mathbf{A}\\\mathbf{B}\end{bmatrix}\mathbf{W}_M \right\rangle$$
where $\mathbf{a}, \mathbf{b} \in \mathbb{R}^d, \mathbf{A}, \mathbf{B} \in \mathbb{R}^{d \times d}, \mathbf{W} \in \mathbb{R}^{2d\times d}$, and $\mathbf{W}_M \in \mathbb{R}^{2d\times  d}$.

Now, the parameters to learn are $\mathbf{W}, \mathbf{W}_M$ and the matrices $\mathbf{X}$ for all leaf vectors $\mathbf{x}$.

### Other models

- RNTN (Recursive Neural Tensor Network) (Socher et al. 2013): similar to MV-RNNs, but with a fixed number of parameters to learn.
- FCN (Forest Convolutional Network) (Le & Zuidema 2015): extends RNN to sets of trees.
- Tree LSTM (Tai, Socher, & Manning 2015): inspired by LSTMs and RNNs. Uses an LSTM-like architecture over trees, rather than sequences.


## Matrix representations of predicates

Taking inspiration from Montague semantics, these models represent relational terms (verbs, adjectives) as linear functions on vectors, encoded as matrices. I discuss two models that apply this idea to a restricted set of syntactic structures.

### Adjective-noun pairs (Baroni & Zamparelli 2010)

In an adjective-noun sequence (e.g., 'green chair'), represent the adjective as a matrix and the noun as a vector. The adjective acts as a linear function of nouns, so the composition function is defined as such:
$$ \text{comp}(\mathbf{A}, \mathbf{n}, -) = \mathbf{nA} $$
where $\mathbf{n} \in \mathbb{R}^d, \mathbf{A} \in \mathbb{R}^{d \times d}$

Baroni & Zamparelli (2010) assume the noun vectors are given, and learn adjective vectors. To do so, they treat each horizontal slice of $\mathbf{A}$ as a linear regression model:
$$ y_i = \mathbf{A}_{[i,\cdot]}x. $$
So each adjective matrix $\mathbf{A} \in \mathbb{R}^{d \times d}$ is estimated by $d$ linear regressions.

For each adjective, the regression model uses as training data a distributional adjective matrix constructed as follows:
1. obtain a cooccurance matrix for the adjective
2. transform the raw counts to Local Mutual Information (LMI) scores
3. apply Singular Value Decomposition (SVD) to reduce dimensionality


### Transitive verbs (Grefenstette & Sadrzadeh 2010, 2011)

In a transitive sentence, represent the subject and object as vectors, and the verb as a matrix. The meaning of the whole sentence is the component-wise product of verb with the outer product of the subject and object.

$$ \overrightarrow{sub~verb~obj} = \underline{verb} \odot (\overrightarrow{sub} \otimes \overrightarrow{obj}) $$

where the outer product of two vectors $\mathbf{u}, \mathbf{v} \in \mathbb{R}^d$ is:
$$ [\mathbf{u} \otimes \mathbf{v}]_{[i,j]} := u_iv_j. $$
This gives a $d\times d$ matrix
$$ \mathbf{u} \otimes \mathbf{v} = \begin{bmatrix}
    u_1v_1 & \cdots & u_1v_d \\
    \vdots & \ddots & \vdots \\
    u_dv_1 & \cdots & u_dv_d
\end{bmatrix}.
$$

We can think of this as two separate composition functions, one between vectors which transforms them into matrices, and one between matrices:

$$ \text{comp}(\mathbf{u}, \mathbf{v}, -) = \mathbf{u} \otimes \mathbf{v} $$
$$ \text{comp}(\mathbf{A}, \mathbf{B}, -) = \mathbf{A} \odot \mathbf{B} $$

So the meaning of the sentence is computed by
$$ \text{comp}(\overrightarrow{verb}, \text{comp}(\overrightarrow{sub}, \overrightarrow{obj}, -), -). $$

Grefenstette & Sadrzadeh do not learn verb matrices, but rather experiment with three ways of generating verb matrices from verb vectors. Let $\mathbf{v}$ be a verb vector. Consider defining the matrix $\mathbf{V}$ in three ways:
1. Embed $\mathbf{v}$ as the diagonal of $\mathbf{V}$, with all other entries set to 0.
$$ \mathbf{V} = \begin{bmatrix}
    v_1 & 0 & 0 & 0 \\
    0 & v_2 & 0 & 0 \\
    0 & 0 & \ddots & 0 \\
    0 & 0 & 0 & v_d \\
\end{bmatrix} $$
2. Embed $\mathbf{v}$ as the diagonal of $\mathbf{V}$, with all other entries set to 1.
$$ \mathbf{V} = \begin{bmatrix}
    v_1 & 1 & 1 & 1 \\
    1 & v_2 & 1 & 1 \\
    1 & 1 & \ddots & 1 \\
    1 & 1 & 1 & v_d \\
\end{bmatrix} $$
3. Take the outer product of the verb with itself.
$$ \mathbf{V} = \mathbf{v} \otimes \mathbf{v} = \begin{bmatrix}
    v_1v_1 & \cdots & v_1v_d \\
    \vdots & \ddots & \vdots \\
    v_dv_1 & \cdots & v_dv_d
\end{bmatrix} $$
